# Notebook pour l'implémentation d'une première pipeline

Le but de ce notebook est de faire une première pipeline qui, à partir d'un ensemble typique de documents, génère la demande de financements souhaitée.

## Load documents

In [1]:
# Here, load the documents in python
# Function to read the questions inside files with .docx extension contained in a folder
def Read_Questions_From_docx (PathFolderSource, PathForOutputsAndLogs):
# This program reads the content of files with .docx extension contained in a folder
# It uses python-docx 1.1.2 to manipulate Word documents : .docx only but not .doc so you need first to type "pip install python-docx" in your terminal
# It identifies the questions for AI from the other information by looking for the tag "SQAI " at the beginning of the question
# and for the tag "EQAI" at the end of the question (a question can have several paragraphs)
# SQAI stands for Start of the Question for AI and EQAI stands for End of the Question for AI
# The ouptput of this function is double :
# 1°) return a dictionary containing the questions for AI : Key= "NameOfFile - Question Number" and Value = Text of the question
# 2°) create in a folder a new version of each document that has been read, where below each question,
#  is added the same Key "NameOfFile - Question Number"
# After the answers are created, It will allow to insert the answers at the right place just below the corresponding question in the documents
# The user will then be able to see and modify in each document the original question and the answer given by the AI
# The function also logs errors in a file named "logs-IA_for_Asso.txt" in the folder "PathForOutputsAndLogs"


    #activate logging of errors in a txt file
    from datetime import datetime
    import logging
    logging.basicConfig(filename=PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt')

    #Create a list of path to all the files (no hidden files) contained in the folder “PathFolderSource” 
    import glob
    FilesWithPath = []
    for file in glob.glob(PathFolderSource +'*.*'):
        FilesWithPath.append(file)

    ItIsAQuestion = False # Tag that indicates if the current paragraph is inside a question for AI
    QuestionNumber = 0 # Number of the question for AI
    TheTextofTheQuestion = '' # Text of a question for AI
    DictQuestions = {} #initialise the dictionnary of questions
    # read content of the files, only if they are .docx (extension to other file types possible with the match - case)
    for file in FilesWithPath:
        TheExtension = file [-4:] 
        match TheExtension:
            case 'docx':
                try:
                    f = open(file, 'rb')
                    document = Document(f)
                    NameOfDocument = file.split('/')[-1] # Name of the file without the path will be used in the Key of the dictionnary
                    for docpara in document.paragraphs:
                        if (docpara.text != ''): # we don't want to add empty paragraphs
                            if(docpara.text[:4]=='SQAI'): # if first characters are "SQAI", then it is the start of a question for AI
                                ItIsAQuestion = True
                                TheTextofTheQuestion = docpara.text[4:]# eliminate the 4 first characters which are the TAG "SQAI"
                            else:
                                if (ItIsAQuestion): # if we are inside a question
                                    TheTextofTheQuestion = TheTextofTheQuestion + ". "+ docpara.text
                            if (docpara.text[-4:]=='EQAI'): # if the end of the paragraph is EQAI, then it is the end of the question
                                ItIsAQuestion = False
                                TheTextofTheQuestion = TheTextofTheQuestion[:-4]# eliminate the 4 last characters which are the TAG "EQAI"
                                QuestionNumber += 1
                                QuestionUI = NameOfDocument + ' - ' + str(QuestionNumber)
                                DictQuestions[QuestionUI] = TheTextofTheQuestion #add the question to the dictionary with a Unique ID
                                docpara.text = docpara.text + '\n' + QuestionUI
                                #TO DO AFTER : manager les infos entre les questions si on doit les fournir à l'IA
                                #TO DO AFTER : dans un dictionaire de complément d'infos
                                #TO DO AFTER : Gérer les numérotations indentées qui sous-divisent les questions ?
                                #TO DO AFTER : Gérer les tableaux ?
                                #TO DO AFTER : Gérer la résistance à l'erreur = début SQAI mais manque fin EQAI ou inverse

                    document.save(PathForOutputsAndLogs+ r'/' + NameOfDocument)
                except IOError:
                        MessageError = str(datetime.now()) + ' Error encountered when reading Word docx file ' + file
                        logging.error(MessageError)
                        print(MessageError)
                finally:        
                    f.close()

            case '.doc':
                print('Fichier DOC')# OPEN QUESTION: do we consider reading .doc files ?
            case _:
                print('Fichier non pris en charge')
                #OPEN QUESTION: do we consider reading other types of files below ?
                #'rtf', 'pdf', 'xls', 'xlsx', 'csv', 'ppt', 'pptx',
                #'odc','odf', 'odg', 'odm', 'odp', 'ods','odt', 'odx'
                # WE SHOULD CHECK ALL EXTENSIONS OF THE FILES CONTAINED IN THE FOLDER 
                # AND PROMPT A MESSAGE IF EXTENSION NOT MANAGED
    print('End of the read program')
    return DictQuestions

# Function to write the answer below each question inside files with .docx extension contained in a folder
def Write_Answers_in_docx (PathFolderSource, DictonaryOfAnswers, PathForOutputsAndLogs):
# The main program has already submitted each question to the AI 
# and filled the "DictonaryOfAnswers" with the answers to the questions 
# The "DictonaryOfAnswers" has the same Key "NameOfFile - Question Number" as the "DictonaryOfQuestions"
# Then the main program will call the "Write_Answers_in_docx" function to write the answers 
# from the he "DictonaryOfAnswers" into the documents themselves
# As the read function has already placed the key of the question below the question, 
# this function will just have to find the key below the question and replace ti by the answer, back in the docx file 
# It will also remove the SQAI and EQAI tags from the questions


     #activate logging of errors in a txt file
    from datetime import datetime
    import logging
    logging.basicConfig(filename=PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt')

    #Create a list of path to all the files (no hidden files) contained in the folder “PathFolderSource” 
    import glob
    FilesWithPath = []
    for file in glob.glob(PathFolderSource +'*.*'):
        FilesWithPath.append(file)
    #FilesWithPath.remove(PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt') # remove the log file from the list of files to be read
    #TO DO AFTER : manage the case where the log file is not in the folder
    for file in FilesWithPath:
        TheExtension = file [-4:] 
        match TheExtension:
            case 'docx':
                try:
                    f = open(file, 'rb')
                    document = Document(f)
                    NameOfDocument = file.split('/')[-1] # Name of the file without the path will be used in the Key of the dictionnary

                    # for each key of the dictionary, corresponding to the document
                    # find the key in the document and replace it by the answer
                    # As the key was below the question, this puts the answer just below the question
                    # if the key is not found, log an error

                    # Create a subset of the dictionary corresponding to the document opened
                    Dict_Of_Answers_of_the_Document = dict(filter(lambda item: item[0].split(' - ')[0] == NameOfDocument, DictonaryOfAnswers.items()))
                    print(Dict_Of_Answers_of_the_Document) # The answer dictionnary for the document
                    for docpara in document.paragraphs:
                        for key, value in Dict_Of_Answers_of_the_Document.items():
                            if key in docpara.text:
                                docpara.text = docpara.text.replace(key, value)
                                # Dict_Of_Answers_of_the_Document.pop(key) # remove the key from the dictionnary when it has been found

                    # Now, we suppress the tags SQAI and EQAI from the questions
                    for docpara in document.paragraphs:
                        if "SQAI" in docpara.text:
                            docpara.text = docpara.text.replace("SQAI", "")
                        if "EQAI" in docpara.text:
                            docpara.text = docpara.text.replace("EQAI", "")

                    # We create a new version of the document with the answers
                    document.save(PathForOutputsAndLogs+ r'/' + NameOfDocument[:-4] + "_with_answers.docx")
                except IOError:
                        MessageError = str(datetime.now()) + ' Error encountered when opening for writing the Word docx file ' + file
                        logging.error(MessageError)
                        print(MessageError)
                finally:        
                    f.close()

    print('End of the write program')
    return



# *******************************MAIN PROGRAM*************************************
# Settings for the path files
Path_where_we_put_Outputs = r'/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python/Dev/Dev IA Asso/Pour les logs/' 
Folder_where_the_files_are = r'/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python/Dev/Dev IA Asso/LesFilesA Lire/'
from docx import Document # import de python-docx

# Read the questions in the files and put them into a dictionnary
The_Dict_Of_Questions = Read_Questions_From_docx (Folder_where_the_files_are, Path_where_we_put_Outputs)

# TO DO : The main programm should then call the AI to answer the questions of the dictionary "The_Dict_Of_Questions"
# and put the answers into a "dictionnary of answers" with the same keys (key of question = key of answer)

# For the moment, we create a dictionary of answers with the same keys as the dictionary of questions
# by just taking the question as the answer we just put "ANSWER TO: " + the question

for key, value in The_Dict_Of_Questions.items():
        The_Dict_Of_Answers = {key:  value for key,  value in The_Dict_Of_Questions.items()}
for key, value in The_Dict_Of_Answers.items():
        The_Dict_Of_Answers[key] = '\rn' + 'ANSWER TO: ' + value
# Write the answers into the docx files just below the questions
Write_Answers_in_docx (Path_where_we_put_Outputs, The_Dict_Of_Answers, Path_where_we_put_Outputs)

#********************************END OF MAIN PROGRAM********************************


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python/Dev/Dev IA Asso/Pour les logs/logs-IA_for_Asso.txt'

## (Optional in the beginning) Chunk and embedd documents

Chunking and embedding documents is a way to implement a RAG (Retrieval Augmented Generation). 

To learn about this concept, you can check the following links :

Here are also useful resources to implement a RAG in python using langchain :



In [ ]:
#LOAD-DOCUMENTS = READ QUESTIONS IN NEW AAPs AND WRITE ANSWERS PROPOSED BY AI INTO NEW AAPs
# ========================================================================================================================================================
# READ EMPTY AAPs : this program has a function that reads the questions in  .docx files contained in a folder
# and moves the questions into a dictionary with a unique ID (UID) for each question
# This UID is also writen below the question in the .docx files
# The questions are identified by tags at the beginning and at the end of the question in the docx files.
# this function also reads tables in the .docx files to retreive the questions contained in the tables (no tag necessary)
# only 3 types of standard tables are managed and the other types of tables are ignored
# the UID is written into the cells of the tables which are waiting for an answer
# =======================================================================================================================================================
# WRITE DOCUMENTS TO FILL ANSWERS IN EMPTY AAPs : This program has also a function that writes the answers to the questions into the .docx files
# using a dictionnary of answers associated to the same UID as the questions
# So, the answers are written into the .docx files, below the questions or inside the cells of the tables 
# 
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ TESTS & IMPROVEMENTS NEEDED @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# 1°) faire des tests avec vrais AAP nouveaux pour fiabiliser la lecture des tableaux
# 2°) Distinguer les type de doc lus: AAP pour AAP nouveau et AAPE + PP pour context IA et activer une pure lecture tabelau pour AAPE+PP
# 3°) Améliorer la détection des tables matricielles en contrôlant que toute la première ligne et toute la première colonne sont non vides
# 4°) Envoyer vraiment à l'IA les question des tableau matriciels pour vérifier la compréhension
# 5°) Mettre les bonnes valeurs des tags de questions defined by Kristin
# 6°) Gérer la distinction des tags généraux et des tags projets et envoyer vers 2 dictionnaires distincts ??
# 8°) Improve file error management : file not in the folder, not readable, not writable, not closed, not found
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END TESTS & IMPROVEMENTS NEEDED @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


#@@@@@@@@@@@@@@@@@@@@@@@@@@@ "READ QUESTIONS FROM NEW AAP" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Function to read the questions inside files with .docx extension contained in a folder using tags
def Read_Questions_From_docx (PathFolderSource, PathForOutputsAndLogs):
# This program reads the content of files with .docx extension contained in a folder
# It uses python-docx 1.1.2 to manipulate Word documents : .docx only but not .doc so you need first to type "pip install python-docx" in your terminal
# It identifies the questions from the other information by looking for the tag TagStartGeneralQuestion at the beginning of the question
# and for the tag TagEndGeneralQuestion at the end of the question (a question can have several paragraphs)
# TagStartGeneralQuestion indicates the Start of the Question and TagEndGeneralQuestion indicates the End of the Question
# The ouptput of this function is double :
# 1°) return a dictionary containing the questions for AI : Key= "NameOfFile - Unique ID" and Value = Text of the question
# 2°) create in a folder a new version of each document that has been read, where below each question,
#  is added the same Key "NameOfFile - Unique ID"
# After the answers are created, It will allow to insert the answers at the right place just below the corresponding question in the documents
# The user will then be able to see and modify in each document the original question and the answer given by the AI
# The function also logs errors in a file named "logs-IA_for_Asso.txt" in the folder "PathForOutputsAndLogs"

    # for unique ID creation
    import uuid

 
    #activate logging of errors in a txt file
    from datetime import datetime
    import logging
    logging.basicConfig(filename=PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt')

    #Create a list of path to all the files (no hidden files) contained in the folder “PathFolderSource” 
    import glob
    FilesWithPath = []
    for file in glob.glob(PathFolderSource +'*.*'):
        FilesWithPath.append(file)

    # initialize variables
    ItIsAQuestion = False # Tag that indicates if the current paragraph is inside a question
    TheTextofTheQuestion = '' # Text of a question
    DictQuestions = {} #initialise the dictionnary of questions
    TagStartGeneralQuestion = '<gquestion>' # Tag that indicates the start of a general question (information about the NGO,..)
    TagEndGeneralQuestion = '<gquestion/>' # Tag that indicates the end of a general question
    LenTagStartGeneralQuestion = len(TagStartGeneralQuestion) # length of the tag
    LenTagEndGeneralQuestion = len(TagEndGeneralQuestion) # length of the tag
    TagSartProjectQuestion = '<pquestion>' # Tag that indicates the start of a project question (information about a project proposed by the NGO)
    TagEndProjectQuestion = '<pquestion/>' # Tag that indicates the end of a project question

    # read content of the files, only if they are .docx (extension to other file types possible with the match - case)
    for file in FilesWithPath:
        TheExtension = file [-4:] 
        match TheExtension:
            case 'docx':
                try:
                    f = open(file, 'rb')
                    document = Document(f)
                    NameOfDocument = file.split('/')[-1] # Name of the file without the path will be used in the Key of the dictionnary

                    # here below, we retrieve the questions included in the tables of the document, 
                    # We manage 3 standard types of tables and other types of tables are ignored

                    # *****************type 0 : table with only one column
                    # the first row is the question and the row below is waiting for the answer
                    # the row below must be empty (if it contains additional information, the function will not manage it properly)
                    # the must be only 1 empty row below the question (if it is not the cas, the function will not manage it properly)
                    # the "question" retrieved is then the content of not empty row and the UID (for the answer) is written in the empty row

                    # *****************type 1 : table with two columns
                    # the first column is the question and the second column is for the answer
                    # the second column is generally empty but can sometimes contain additional information
                    # the "question" retrieved is then the content of 1srt column concatenated with de content of 2nd column

                    # ******************type 2 : table with more than two columns and the first row not empty and the first column not empty
                    # this is a standard matrix table with information in rows and columns, 
                    # and answers awaited at the crossing of rows and columns
                    # the "question" retrieved is then the content of row 0 column 0 (title of the table)
                    # concatenated with the content of row 0 column X  (X going from 1 to the max column number)
                    # concatenated with the content of row Y column 0 (Y going from 1 to the max row number)
                    # and the corresponding answer (UID) shall be put in row Y column X
                    # ******************type 2 variant :
                    # generally, only the first row is not empty but 
                    # sometimes, the second row is also not empty = when there are merged cells in the first row 
                    # and the second row is a sub decomposition of the first row (e.g. 1srt row Year and 2nd row Month)
                    # the "question" retrieved is then the content of row 0 column 0 (title of the table)
                    # concatenated with the content of row 0 column X  (X going from 1 to the max column number)
                    # concatenated with the content of row 1 column X  (X going from 1 to the max column number)
                    # concatenated with the content of row Y column 0 (Y going from 1 to the max row number)
                    # and the corresponding answer shall be put in row Y column X
                    for index, table in enumerate(document.tables):
                        NBColumns = len(table.columns)
                        if NBColumns == 1: # it is a "type 0" table 
                            print("Type 0 table")
                            for row in range(len(table.rows)):
                                if table.cell(row, 0).text.lstrip(" ") != '':# if the cell is not empty, it is a "question"
                                    ItIsAQuestion =True
                                    TheTextofTheQuestion = table.cell(row, 0).text 
                                    QuestionUI = NameOfDocument + ' - ' + uuid.uuid4().hex # create a unique ID for the question
                                    DictQuestions[QuestionUI] = TheTextofTheQuestion #add the question to the dictionary with a Unique ID
                                if ItIsAQuestion and row>=1 and table.cell(row, 0).text.lstrip(" ") == '' and table.cell(row-1, 0).text.lstrip(" ") != '':
                                    # if the cell is empty and the previous cell is not empty, the current cell is waiting for the answer of the question of the previous cell
                                    # so we write the UID of the previous question into the cell
                                    # ItIsAQuestion is tested to manage the case of several empty rows below a question
                                    table.cell(row, 0).text = QuestionUI
                                    ItIsAQuestion =False # to manage the case where the table has more than 1 empty row below a "question"
                                print("in row = "+str(row)+" and Col = "+str(0)+", the content is "+table.cell(row, 0).text, end='\n')
                       
                        if NBColumns == 2: # it is a "type 1" table 
                            print("Type 1 table")
                            for row in range(len(table.rows)):
                                if table.cell(row, 0).text.lstrip(" ") != '':
                                    TheTextofTheQuestion = table.cell(row , 0).text + ' ' + table.cell(row , 1).text # concatenate the 2 columns
                                    QuestionUI = NameOfDocument + ' - ' + uuid.uuid4().hex # create a unique ID for the question
                                    DictQuestions[QuestionUI] = TheTextofTheQuestion #add the question to the dictionary with a Unique ID
                                    table.cell(row, 1).text = QuestionUI # write the UID in the second column
                                print("in row = "+str(row)+" and Col = "+str(0)+", the content is "+table.cell(row, 0).text, end='\n')
                                print("in row = "+str(row)+" and Col = "+str(1)+", the content is "+table.cell(row, 1).text, end='\n')

                        # For more than 2 columns, we consider only the case of a "type 2" table (matrix table)
                        #  when row 0 is not empty and col 0 is not empty and we ignore the other cases
                        # we test uniquely the first row 2nd col (row = 0 col = 1) and the first column 2nd row (row 1 & col=0)
                        if (NBColumns >2) and (table.cell(0, 1).text.lstrip(" ") != '') and (table.cell(1, 0).text.lstrip(" ") != ''):
                            print("Type 2 table")
                            # A FAIRE : Gérer les cas où la 2ème ligne n'est pas vide et est une sous décomposition de la 1ère ligne
                            # ---------------- CASE TYPE 2 STANDARD WITH ONLY 1 ROW OF TITLES----------------
                            if (table.cell(1, 1).text.lstrip(" ") == ''): # if the second Row is empty = it is a standard matrix table Type2
                                for row in range(1, len(table.rows) ):   # From second row (1) to max row 
                                    for col in range(1, len(table.columns) ):  #  From second col (1) to max col
                                        if table.cell(row, col).text.lstrip(" ") == '':
                                            TheTextofTheQuestion = table.cell(0,0).text + " " + table.cell(0,col).text + " " + table.cell(row, 0).text
                                            QuestionUI = NameOfDocument + ' - ' + uuid.uuid4().hex
                                            DictQuestions[QuestionUI] = TheTextofTheQuestion #add the question to the dictionary with a Unique ID
                                            # question to the dictionary with a Unique ID
                                            table.cell(row, col).text = QuestionUI # put UID in the cell of the table
                            # ---------------- CASE TYPE 2 VARIANT WITH 2 ROWS OF TITLES ----------------
                            # the second row is also not empty = when there are merged cells in the first row 
                            # and the second row is a sub decomposition of the first row (e.g. 1srt row Year and 2nd row Month)
                            if (table.cell(1, 1).text.lstrip(" ") != ''): # if the second Row is not empty = it is a variant matrix table Type2
                                for row in range(2, len(table.rows) ):   # From third row (2) to max row 
                                    for col in range(1, len(table.columns) ):  #  From second col (1) to max col
                                        if table.cell(row, col).text.lstrip(" ") == '':
                                            TheTextofTheQuestion = table.cell(0,0).text + " " + table.cell(0,col).text + " " + table.cell(1,col).text + " " + table.cell(row, 0).text
                                            QuestionUI = NameOfDocument + ' - ' + uuid.uuid4().hex
                                            DictQuestions[QuestionUI] = TheTextofTheQuestion #add the question to the dictionary with a Unique ID
                                            # question to the dictionary with a Unique ID
                                            table.cell(row, col).text = QuestionUI # put UID in the cell of the table
                        print("Nbr of columns = "+str(len(table.columns))+" and Nbr of rows =  "+str(len(table.rows)), end='\n')
                        for row in range(len(table.rows)):
                            for col in range(len(table.columns)):
                                print("in row = "+str(row)+" and Col = "+str(col)+", the content is "+table.cell(row, col).text, end='\n')
                        print()
                    print()

                    # then here, we retrieve the questions identified by tags TagStartGeneralQuestion and TagEndGeneralQuestion in the full text of the document
                    for docpara in document.paragraphs:
                        if (docpara.text != ''): # we don't want to add empty paragraphs
                            if(docpara.text[:LenTagStartGeneralQuestion]==TagStartGeneralQuestion): # if first characters are TagStartGeneralQuestion, then it is the start of a question
                                ItIsAQuestion = True
                                TheTextofTheQuestion = docpara.text[LenTagStartGeneralQuestion:]# eliminate the n first characters which are the TAG TagStartGeneralQuestion
                            else:
                                if (ItIsAQuestion): # if we are inside a question
                                    TheTextofTheQuestion = TheTextofTheQuestion + ". "+ docpara.text
                            if (docpara.text[-LenTagEndGeneralQuestion:]==TagEndGeneralQuestion): # if the end of the paragraph is TagEndGeneralQuestion, then it is the end of the question
                                ItIsAQuestion = False
                                TheTextofTheQuestion = TheTextofTheQuestion[:-LenTagEndGeneralQuestion]# eliminate the n last characters which are the TAG TagEndGeneralQuestion
                                QuestionUI = NameOfDocument + ' - ' + uuid.uuid4().hex
                                DictQuestions[QuestionUI] = TheTextofTheQuestion #add the question to the dictionary with a Unique ID
                                docpara.text = docpara.text + '\n' + QuestionUI
                                #TO DO AFTER : manager les infos entre les questions si on doit les fournir à l'IA
                                #TO DO AFTER : dans un dictionaire de complément d'infos
                                #TO DO AFTER : Gérer les numérotations indentées qui sous-divisent les questions ?
                                #TO DO AFTER : Gérer les tableaux ?
                                #TO DO AFTER : Gérer la résistance à l'erreur = début TagStartGeneralQuestion mais manque fin TagEndGeneralQuestion ou inverse

                    document.save(PathForOutputsAndLogs+ r'/' + NameOfDocument)
                except IOError:
                        MessageError = str(datetime.now()) + ' Error encountered when reading Word docx file ' + file
                        logging.error(MessageError)
                        print(MessageError)
                finally:        
                    f.close()

            case '.doc':
                print('Fichier DOC')# OPEN QUESTION: do we consider reading .doc files ?
            case _:
                print('Fichier non pris en charge')
                #OPEN QUESTION: do we consider reading other types of files below ?
                #'rtf', 'pdf', 'xls', 'xlsx', 'csv', 'ppt', 'pptx',
                #'odc','odf', 'odg', 'odm', 'odp', 'ods','odt', 'odx'
                # WE SHOULD CHECK ALL EXTENSIONS OF THE FILES CONTAINED IN THE FOLDER 
                # AND PROMPT A MESSAGE IF EXTENSION NOT MANAGED
    print('End of the read program')
    return DictQuestions
#@@@@@@@@@@@@@@@@@@@@@@@ END OF "READ QUESTIONS FROM NEW AAP" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ "WRITE ANSWERS INTO NEW AAP" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Function to write the answer below each question inside files with .docx extension contained in a folder
def Write_Answers_in_docx (PathFolderSource, DictonaryOfAnswers, PathForOutputsAndLogs):
# The main program has already submitted each question to the AI 
# and filled the "DictonaryOfAnswers" with the answers to the questions 
# The "DictonaryOfAnswers" has the same Key "NameOfFile - Unique ID" as the "DictonaryOfQuestions"
# Then the main program will call the "Write_Answers_in_docx" function to write the answers 
# from the he "DictonaryOfAnswers" into the documents themselves
# As the read function has already placed the key of the question below the question, 
# this function will just have to find the key below the question and replace ti by the answer, back in the docx file 
# It will also remove the TagEndGeneralQuestion and TagEndGeneralQuestion tags from the questions


     #activate logging of errors in a txt file
    from datetime import datetime
    import logging
    logging.basicConfig(filename=PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt')

    # initialize variables
    TagStartGeneralQuestion = '<gquestion>' # Tag that indicates the start of a general question (information about the NGO,..)
    TagEndGeneralQuestion = '<gquestion/>' # Tag that indicates the end of a general question
    TagSartProjectQuestion = 'SQPR' # Tag that indicates the start of a project question (information about a project proposed by the NGO)
    TagEndProjectQuestion = 'EQPR' # Tag that indicates the end of a project question


    #Create a list of path to all the files (no hidden files) contained in the folder “PathFolderSource” 
    import glob
    FilesWithPath = []
    for file in glob.glob(PathFolderSource +'*.*'):
        FilesWithPath.append(file)
    #FilesWithPath.remove(PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt') # remove the log file from the list of files to be read
    #TO DO AFTER : manage the case where the log file is not in the folder
    for file in FilesWithPath:
        TheExtension = file [-4:] 
        match TheExtension:
            case 'docx':
                try:
                    f = open(file, 'rb')
                    document = Document(f)
                    NameOfDocument = file.split('/')[-1] # Name of the file without the path will be used in the Key of the dictionnary

                    # for each key of the dictionary, corresponding to the document
                    # find the key in the document and replace it by the answer
                    # As the key was below the question, this puts the answer just below the question
                    # if the key is not found, log an error

                    # Create a subset of the dictionary corresponding to the document opened
                    Dict_Of_Answers_of_the_Document = dict(filter(lambda item: item[0].split(' - ')[0] == NameOfDocument, DictonaryOfAnswers.items()))
                    print(Dict_Of_Answers_of_the_Document) # The answer dictionnary for the document
                    
                    # Now, we replace the keys by the answers in the full text of the document
                    for docpara in document.paragraphs:
                        for key, value in Dict_Of_Answers_of_the_Document.items():
                            if key in docpara.text:
                                docpara.text = docpara.text.replace(key, value)
                                # Dict_Of_Answers_of_the_Document.pop(key) # remove the key from the dictionnary when it has been found

                    # then, we replace the keys by the answers in the tables of the document
                    for index, table in enumerate(document.tables):
                        for key, value in Dict_Of_Answers_of_the_Document.items():
                            for row in range(len(table.rows)):
                                for col in range(len(table.columns)):
                                   if key in table.cell(row, col).text:
                                       table.cell(row, col).text = table.cell(row, col).text.replace(key, value)


                    # Now, we suppress the tags TagStartGeneralQuestion and TagEndGeneralQuestion from the questions
                    for docpara in document.paragraphs:
                        if TagStartGeneralQuestion in docpara.text:
                            docpara.text = docpara.text.replace(TagStartGeneralQuestion, "")
                        if TagEndGeneralQuestion in docpara.text:
                            docpara.text = docpara.text.replace(TagEndGeneralQuestion, "")

                    # We create a new version of the document with the answers
                    document.save(PathForOutputsAndLogs+ r'/' + NameOfDocument[:-4] + "_with_answers.docx")
                except IOError:
                        MessageError = str(datetime.now()) + ' Error encountered when opening for writing the Word docx file ' + file
                        logging.error(MessageError)
                        print(MessageError)
                finally:        
                    f.close()

    print('End of the write program')
    return
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF "WRITE ANSWERS INTO NEW AAP" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ MAIN PROGRAM @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Settings for the path files
Path_where_we_put_Outputs = r'/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python yc Dev D4G/3 - Dev IA Asso/Pour les logs/' 
Folder_where_the_files_are = r'/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python yc Dev D4G/3 - Dev IA Asso/LesFilesA Lire/'
from docx import Document # import de python-docx

#tuple(c.text for c in r.cells) for r in table.rows


# Read the questions in the files and put them into a dictionnary
The_Dict_Of_Questions = Read_Questions_From_docx (Folder_where_the_files_are, Path_where_we_put_Outputs)

# TO DO : The main programm should then call the AI to answer the questions of the dictionary "The_Dict_Of_Questions"
# and put the answers into a "dictionnary of answers" with the same keys (key of question = key of answer)

# For the moment, we create a dictionary of answers with the same keys as the dictionary of questions
# by just taking the question as the answer we just put "ANSWER TO: " + the question

for key, value in The_Dict_Of_Questions.items():
        The_Dict_Of_Answers = {key:  value for key,  value in The_Dict_Of_Questions.items()}
for key, value in The_Dict_Of_Answers.items():
        The_Dict_Of_Answers[key] = ' ANSWER TO: ' + value
# Write the answers into the docx files just below the questions
Write_Answers_in_docx (Path_where_we_put_Outputs, The_Dict_Of_Answers, Path_where_we_put_Outputs)

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF MAIN PROGRAM @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



!! It is important to note that while RAG is a common way to provide LLMs with context, specific methods can be used for this project. For instance, maybe that all documents have an "information about x" section that can be directly retrieved with regex methods to provide the model with.

For regex methods, you can find documentation here :


In [ ]:
# Here split the document into chunks

In [ ]:
# Here embed those chunks

In [ ]:
# (Optional) Here you can store those embedded chunks into a vector store

## call a large language model via an API (e.g. Mistral API call - use free tiers)

Here we're gonna call a model (and pass him the context if already implemented before)

Some links you can check to learn more if you don't know how it works :

Langchain (one of the classic tools for this kind of task)


<b>To run a model locally</b>

With Ollama :

With huggingface : 

In [ ]:
"""
Here, first write your credentials for API call (don't push it on git !! Use environment variables)
or load the model in the notebook kernel if you want to use a model locally
"""

In [ ]:
"""
Then, implement API calling (langchain chain + prompt engineering)
You can divide the whole process in several sub-questions if the model can't take enough context at once,
or if it does not perform well enough.
"""

## (Very very optional) Implement a langgraph to enhance generation performances with agentic behavior

This step should not be necessary but once everything else is set up, you can play with it.

Documentation : 

In [ ]:
# Langgraph implementation